In [1]:
import sys
sys.path.insert(0,"/home/albert7a/lib/python")


## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar
from datetime import date
import yaml
import io
import json
import os
import warnings
warnings.filterwarnings('ignore')


In [2]:
jsonfile="medred_201303.json"

In [4]:
sourcefile=open(jsonfile,'rU')
infos=json.load(sourcefile)
nb_profilesEN4=len(infos)

print nb_profilesEN4

125


In [5]:
def open_natl60():
        print 'Opening NATL60'
        #NATL60
        tfiles="/media/extra/DATA/NATL60/NATL60-CJM165-S/TS/NATL60-CJM165_y????m??d??.1d_TS.nc"

        ##Open NATL60 files to get boundaries of domain

        dsN = xr.open_mfdataset(tfiles,concat_dim='time_counter',decode_times=False, chunks={'deptht':1 ,'time_counter':10})

        latN = dsN.nav_lat
        lonN = dsN.nav_lon
        timN = dsN.time_counter
        return dsN,latN,lonN,timN


dsN,latN,lonN,timN = open_natl60()

Opening NATL60


In [7]:
def process(jsonfile,infos,prof,dsN,latN,lonN,timN,step):
        ''' Make all the steps needed to create the final netcdf files for one profile
        '''
        list_profiles = infos.keys()
        reference =  str(list_profiles[prof])
        print 'Processing profile ', reference

        lat_prof = int(infos[list_profiles[prof]]['latitude'])
        lon_prof = int(infos[list_profiles[prof]]['longitude'])
        date_prof = str(infos[list_profiles[prof]]['date'])
        file_prof = str(infos[list_profiles[prof]]['file'])
        prof_prof = int(infos[list_profiles[prof]]['profile no'])

        check=check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
        if check == 0.:
            profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time = model_mean_percent_profile(file_prof,prof_prof,dsN,latN,lonN,timN,step)

def check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within NATL60 boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)
        tmin=np.min(timN.values)
        tmax=np.max(timN.values)

        date_proft=pd.to_datetime(date_prof)
        tprof=datetime.datetime(int(date_proft.strftime('%Y')),int(date_proft.strftime('%m')),int(date_proft.strftime('%d')),int(date_proft.strftime('%H')),int(date_proft.strftime('%M')))
        tsec_prof=(tprof-datetime.datetime(1958,1,1,0,0)).total_seconds()


        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) & (tmin < tsec_prof < tmax):
                check=0.
#                print "selected profile falls within NATL60-CJM165 boundaries, the program is proceeding"
        else:
                check=1.
#                print "selected profile does not fall within NATL60-CJM165 boundaries, the program is stopping"
        return check


def model_mean_percent_profile(fileEN4,ref_prof,dsN,latN,lonN,timN,step):
    #select NATL60 data at the closest depth and within 0.25 and 15days near the location and date of the ARGO profile
    diren4="/media/extra/DATA/EN4/"
    tfileEN4=diren4+fileEN4

    dsen4=xr.open_dataset(tfileEN4)

    laten4=dsen4['LATITUDE'][ref_prof]
    lonen4=dsen4['LONGITUDE'][ref_prof]
    dayen4=dsen4['JULD'][ref_prof]

    dateen4= pd.to_datetime(str(dayen4.values))
    ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
    tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()

    observation_lon=lonen4
    observation_lat=laten4
    observation_time=tsecen4

    tempen4=dsen4['POTM_CORRECTED'][ref_prof]
    salten4=dsen4['PSAL_CORRECTED'][ref_prof]
    depen4=dsen4['DEPH_CORRECTED'][ref_prof]

    observation_dep=depen4
    observation_temp=tempen4
    observation_salt=salten4

    depN = dsN.deptht
    tempN=dsN.votemper
    saltN=dsN.vosaline

    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    print dep_level[0]
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    #get the corresponding model level
    model_level=np.zeros(dep_level[0])
    model_dep=np.zeros(dep_level[0])
    obsred_dep=np.zeros(dep_level[0])
    obsred_temp=np.zeros(dep_level[0])
    obsred_salt=np.zeros(dep_level[0])
    for z in np.arange(dep_level[0]):
        obsred_dep[int(z)]=observation_dep[int(z)]
        obsred_temp[int(z)]=observation_temp[int(z)]
        obsred_salt[int(z)]=observation_salt[int(z)]
        diff_dep=0*depN.values
        for k in np.arange(len(depN.values)):
            diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
        lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
        model_level[z]=lev[0]
        model_dep[z]=depN.values[lev[0]]

    #Coarse box in which EN4 profile is contained
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])

    def profile_mean_percent(k):

        #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

        model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]

        model_tempBOX_array=model_tempBOX.values
        model_saltBOX_array=model_saltBOX.values
        model_tBOX_array=model_tBOX.values

        # construction d'un nouveau xarray
        d = {}
        d['time_counter'] = ('time_counter',t_dim)
        d['y'] = ('y',y_dim)
        d['x'] = ('x',x_dim)
        d['nav_lat'] = (['y','x'],model_latBOX_array)
        d['nav_lon'] = (['y','x'],model_lonBOX_array)

        d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
        d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
        d['time_counter'] = (['time_counter'], model_tBOX_array)
        dset = xr.Dataset(d)

        latB = dset.nav_lat
        lonB = dset.nav_lon
        model_temperatureB = dset.votemper
        model_salinityB = dset.vosaline
        model_timeB = dset.time_counter

        # selection plus fine des profils

        lon_stacked = lonB.stack(profile=('x', 'y'))
        lat_stacked = latB.stack(profile=('x', 'y'))

        distance_threshold = 0.25
        square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
        is_close_to_observation = square_distance_to_observation < distance_threshold**2

        model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
        model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))

        model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
        model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
        lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
        lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

        model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
        model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
        model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
        model_salt_dask_concat=dask.array.concatenate(model_salt_dask)

        temp_model_mean = model_temp_dask_concat.mean().compute()
        temp_percentile_10= np.percentile(model_temp_dask_concat,10)
        temp_percentile_90= np.percentile(model_temp_dask_concat,90)
        salt_model_mean = model_salt_dask_concat.mean().compute()
        salt_percentile_10= np.percentile(model_salt_dask_concat,10)
        salt_percentile_90= np.percentile(model_salt_dask_concat,90)

        return lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90

    profil_temp_model_mean=np.zeros(dep_level[0])
    profil_temp_model_percent10=np.zeros(dep_level[0])
    profil_temp_model_percent90=np.zeros(dep_level[0])
    profil_salt_model_mean=np.zeros(dep_level[0])
    profil_salt_model_percent10=np.zeros(dep_level[0])
    profil_salt_model_percent90=np.zeros(dep_level[0])


    for z in np.arange(dep_level[0]):
        lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(model_level[z].astype(int))
        profil_temp_model_mean[z]=temp_model_mean
        profil_temp_model_percent10[z]=temp_percentile_10
        profil_temp_model_percent90[z]=temp_percentile_90
        profil_salt_model_mean[z]=salt_model_mean
        profil_salt_model_percent10[z]=salt_percentile_10
        profil_salt_model_percent90[z]=salt_percentile_90


    return profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time




In [8]:
for prof in np.arange(nb_profilesEN4):
    print time.strftime('%d/%m/%y %H:%M',time.localtime())
    process(jsonfile,infos,prof,dsN,latN,lonN,timN,1)
    print time.strftime('%d/%m/%y %H:%M',time.localtime())
    process(jsonfile,infos,prof,dsN,latN,lonN,timN,3)
    print time.strftime('%d/%m/%y %H:%M',time.localtime())





24/10/17 16:09
Processing profile   A20130325-44028
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:90: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:91: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:92: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:93: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:94: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-pac

24/10/17 16:11
Processing profile   A20130325-44028
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 16:40
24/10/17 16:40
Processing profile   A20130325-44029
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 16:42
Processing profile   A20130325-44029
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 17:10
24/10/17 17:10
Processing profile   A20130330-44595
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 17:13
Processing profile   A20130330-44595
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 17:42
24/10/17 17:42
Processing profile   A20130325-44023
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 17:44
Processing profile   A20130325-44023
selected profile falls within NATL60-CJM165 boundaries, the program i

/home/albert/anaconda2/lib/python2.7/site-packages/numpy-1.11.1-py2.7-linux-x86_64.egg/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


24/10/17 19:18
Processing profile   A20130326-06421
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
80.0
24/10/17 20:39
24/10/17 20:39
Processing profile   A20130325-44025
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 20:41
Processing profile   A20130325-44025
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 21:10
24/10/17 21:10
Processing profile   A20130327-06518
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
106.0
24/10/17 21:17
Processing profile   A20130327-06518
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
106.0
24/10/17 21:26
24/10/17 21:26
Processing profile   A20130327-44558
selected profile falls within NATL60-CJM165 boundaries, the program is proceeding
15.0
24/10/17 21:28
Processing profile   A20130327-44558
selected profile falls within NATL60-CJM165 boundaries, the program

KeyboardInterrupt: 

In [13]:
def process_debug(jsonfile,infos,prof,dsN,latN,lonN,timN,step):
        ''' Make all the steps needed to create the final netcdf files for one profile
        '''
        list_profiles = infos.keys()
        reference =  str(list_profiles[prof])

        lat_prof = int(infos[list_profiles[prof]]['latitude'])
        lon_prof = int(infos[list_profiles[prof]]['longitude'])
        date_prof = str(infos[list_profiles[prof]]['date'])
        file_prof = str(infos[list_profiles[prof]]['file'])
        prof_prof = int(infos[list_profiles[prof]]['profile no'])

        print 'before check ',time.strftime('%H:%M %S',time.localtime())
        check=check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
        print 'after check ',time.strftime('%H:%M %S',time.localtime())
        
        if check == 0.:
            profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time = model_mean_percent_profile_debug(file_prof,prof_prof,dsN,latN,lonN,timN,step)

def check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within NATL60 boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)
        tmin=np.min(timN.values)
        tmax=np.max(timN.values)

        date_proft=pd.to_datetime(date_prof)
        tprof=datetime.datetime(int(date_proft.strftime('%Y')),int(date_proft.strftime('%m')),int(date_proft.strftime('%d')),int(date_proft.strftime('%H')),int(date_proft.strftime('%M')))
        tsec_prof=(tprof-datetime.datetime(1958,1,1,0,0)).total_seconds()


        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) & (tmin < tsec_prof < tmax):
                check=0.
#                print "selected profile falls within NATL60-CJM165 boundaries, the program is proceeding"
        else:
                check=1.
#                print "selected profile does not fall within NATL60-CJM165 boundaries, the program is stopping"
        return check


def model_mean_percent_profile_debug(fileEN4,ref_prof,dsN,latN,lonN,timN,step):
    #select NATL60 data at the closest depth and within 0.25 and 15days near the location and date of the ARGO profile
    diren4="/media/extra/DATA/EN4/"
    tfileEN4=diren4+fileEN4

    print 'before opening EN4 ',time.strftime('%H:%M %S',time.localtime())

    dsen4=xr.open_dataset(tfileEN4)

    print 'after opening EN4 ',time.strftime('%H:%M %S',time.localtime())

    laten4=dsen4['LATITUDE'][ref_prof]
    lonen4=dsen4['LONGITUDE'][ref_prof]
    dayen4=dsen4['JULD'][ref_prof]

    dateen4= pd.to_datetime(str(dayen4.values))
    ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
    tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()

    observation_lon=lonen4
    observation_lat=laten4
    observation_time=tsecen4

    tempen4=dsen4['POTM_CORRECTED'][ref_prof]
    salten4=dsen4['PSAL_CORRECTED'][ref_prof]
    depen4=dsen4['DEPH_CORRECTED'][ref_prof]

    observation_dep=depen4
    observation_temp=tempen4
    observation_salt=salten4

    print 'after load EN4 vars',time.strftime('%H:%M %S',time.localtime())

    depN = dsN.deptht
    tempN=dsN.votemper
    saltN=dsN.vosaline

    print 'after load model vars',time.strftime('%H:%M %S',time.localtime())
    
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    print dep_level[0]
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    #get the corresponding model level
    model_level=np.zeros(dep_level[0])
    model_dep=np.zeros(dep_level[0])
    obsred_dep=np.zeros(dep_level[0])
    obsred_temp=np.zeros(dep_level[0])
    obsred_salt=np.zeros(dep_level[0])
    for z in np.arange(dep_level[0]):
        obsred_dep[int(z)]=observation_dep[int(z)]
        obsred_temp[int(z)]=observation_temp[int(z)]
        obsred_salt[int(z)]=observation_salt[int(z)]
        diff_dep=0*depN.values
        for k in np.arange(len(depN.values)):
            diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
        lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
        model_level[z]=lev[0]
        model_dep[z]=depN.values[lev[0]]
 
    print 'after reduce depth',time.strftime('%H:%M %S',time.localtime())
        
    #Coarse box in which EN4 profile is contained
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])

    print 'after coarse box',time.strftime('%H:%M %S',time.localtime())
    
    def profile_mean_percent(k):


        print 'begin profile deptht k=',k,' ',time.strftime('%H:%M %S',time.localtime())
        
        #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

        model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]

        print 'after box',time.strftime('%H:%M %S',time.localtime())
        
        model_tempBOX_array=model_tempBOX.values
        model_saltBOX_array=model_saltBOX.values
        model_tBOX_array=model_tBOX.values

        print 'after array',time.strftime('%H:%M %S',time.localtime())
        
        # construction d'un nouveau xarray
        d = {}
        d['time_counter'] = ('time_counter',t_dim)
        d['y'] = ('y',y_dim)
        d['x'] = ('x',x_dim)
        d['nav_lat'] = (['y','x'],model_latBOX_array)
        d['nav_lon'] = (['y','x'],model_lonBOX_array)

        d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
        d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
        d['time_counter'] = (['time_counter'], model_tBOX_array)
        dset = xr.Dataset(d)

        latB = dset.nav_lat
        lonB = dset.nav_lon
        model_temperatureB = dset.votemper
        model_salinityB = dset.vosaline
        model_timeB = dset.time_counter

        print 'after xarray',time.strftime('%H:%M %S',time.localtime())
        
        # selection plus fine des profils

        lon_stacked = lonB.stack(profile=('x', 'y'))
        lat_stacked = latB.stack(profile=('x', 'y'))

        distance_threshold = 0.25
        square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
        is_close_to_observation = square_distance_to_observation < distance_threshold**2

        model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
        model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))

        print 'after stack',time.strftime('%H:%M %S',time.localtime())
        
        model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
        model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
        lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
        lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

        print 'after near',time.strftime('%H:%M %S',time.localtime())
        
        model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
        model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
        model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
        model_salt_dask_concat=dask.array.concatenate(model_salt_dask)

        print 'after dask',time.strftime('%H:%M %S',time.localtime())
        
        temp_model_mean = model_temp_dask_concat.mean().compute()
        temp_percentile_10= np.percentile(model_temp_dask_concat,10)
        temp_percentile_90= np.percentile(model_temp_dask_concat,90)
        salt_model_mean = model_salt_dask_concat.mean().compute()
        salt_percentile_10= np.percentile(model_salt_dask_concat,10)
        salt_percentile_90= np.percentile(model_salt_dask_concat,90)

        print 'end profile deptht k=',k,' ',time.strftime('%H:%M %S',time.localtime())
        
        return lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90

    profil_temp_model_mean=np.zeros(dep_level[0])
    profil_temp_model_percent10=np.zeros(dep_level[0])
    profil_temp_model_percent90=np.zeros(dep_level[0])
    profil_salt_model_mean=np.zeros(dep_level[0])
    profil_salt_model_percent10=np.zeros(dep_level[0])
    profil_salt_model_percent90=np.zeros(dep_level[0])


    for z in np.arange(dep_level[0]):
        lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(model_level[z].astype(int))
        profil_temp_model_mean[z]=temp_model_mean
        profil_temp_model_percent10[z]=temp_percentile_10
        profil_temp_model_percent90[z]=temp_percentile_90
        profil_salt_model_mean[z]=salt_model_mean
        profil_salt_model_percent10[z]=salt_percentile_10
        profil_salt_model_percent90[z]=salt_percentile_90


    return profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time



In [10]:
prof=0
print time.strftime('%d/%m/%y %H:%M',time.localtime())
process_debug(jsonfile,infos,prof,dsN,latN,lonN,timN,1)
print time.strftime('%d/%m/%y %H:%M',time.localtime())
process_debug(jsonfile,infos,prof,dsN,latN,lonN,timN,3)
print time.strftime('%d/%m/%y %H:%M',time.localtime())



25/10/17 11:30
before check  25/10/17 11:30
after check  25/10/17 11:30
before opening EN4  25/10/17 11:30
after opening EN4  25/10/17 11:30
after load EN4 vars 25/10/17 11:30
after load model vars 25/10/17 11:30
15.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:100: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:101: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:102: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:103: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:104: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after reduce depth 25/10/17 11:30
after coarse box 25/10/17 11:30
begin profile deptht k= 0   25/10/17 11:30
after box 25/10/17 11:30


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:210: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:211: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:212: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:214: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after array 25/10/17 11:30
after xarray 25/10/17 11:30
after stack 25/10/17 11:30
after near 25/10/17 11:30
after dask 25/10/17 11:30
end profile deptht k= 0   25/10/17 11:30
begin profile deptht k= 7   25/10/17 11:30
after box 25/10/17 11:30


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:220: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:221: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:222: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:223: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after array 25/10/17 11:30
after xarray 25/10/17 11:30
after stack 25/10/17 11:30
after near 25/10/17 11:30
after dask 25/10/17 11:30
end profile deptht k= 7   25/10/17 11:30
begin profile deptht k= 8   25/10/17 11:30
after box 25/10/17 11:30
after array 25/10/17 11:30
after xarray 25/10/17 11:30
after stack 25/10/17 11:30
after near 25/10/17 11:30
after dask 25/10/17 11:30
end profile deptht k= 8   25/10/17 11:30
begin profile deptht k= 9   25/10/17 11:30
after box 25/10/17 11:30
after array 25/10/17 11:31
after xarray 25/10/17 11:31
after stack 25/10/17 11:31
after near 25/10/17 11:31
after dask 25/10/17 11:31
end profile deptht k= 9   25/10/17 11:31
begin profile deptht k= 11   25/10/17 11:31
after box 25/10/17 11:31
after array 25/10/17 11:31
after xarray 25/10/17 11:31
after stack 25/10/17 11:31
after near 25/10/17 11:31
after dask 25/10/17 11:31
end profile deptht k= 11   25/10/17 11:31
begin profile deptht k= 12   25/10/17 11:31
after box 25/10/17 11:31
after array 25/10/17 11:3

In [14]:
prof=51
print time.strftime('%H:%M %S',time.localtime())
process_debug(jsonfile,infos,prof,dsN,latN,lonN,timN,1)
print time.strftime('%H:%M %S',time.localtime())
process_debug(jsonfile,infos,prof,dsN,latN,lonN,timN,3)
print time.strftime('%H:%M %S',time.localtime())



14:54 46
before check  14:54 46
after check  14:54 47
before opening EN4  14:54 47
after opening EN4  14:54 47
after load EN4 vars 14:54 47
after load model vars 14:54 47
80.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:100: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:101: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:102: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:103: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:104: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after reduce depth 14:54 48
after coarse box 14:54 49
begin profile deptht k= 3   14:54 49
after box 14:54 49


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:210: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:211: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:212: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:214: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after array 14:54 51
after xarray 14:54 51
after stack 14:54 51
after near 14:54 51
after dask 14:54 51
end profile deptht k= 3   14:54 52
begin profile deptht k= 6   14:54 52
after box 14:54 52
after array 14:54 52
after xarray 14:54 52


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:220: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:221: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:222: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:223: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/sit

after stack 14:54 52
after near 14:54 52
after dask 14:54 52
end profile deptht k= 6   14:54 54
begin profile deptht k= 8   14:54 54
after box 14:54 54
after array 14:54 54
after xarray 14:54 54
after stack 14:54 54
after near 14:54 54
after dask 14:54 54
end profile deptht k= 8   14:54 55
begin profile deptht k= 10   14:54 55
after box 14:54 55
after array 14:54 55
after xarray 14:54 55
after stack 14:54 55
after near 14:54 55
after dask 14:54 55
end profile deptht k= 10   14:54 57
begin profile deptht k= 12   14:54 57
after box 14:54 57
after array 14:54 57
after xarray 14:54 57
after stack 14:54 57
after near 14:54 57
after dask 14:54 57
end profile deptht k= 12   14:54 58
begin profile deptht k= 14   14:54 58
after box 14:54 58
after array 14:54 58
after xarray 14:54 58
after stack 14:54 58
after near 14:54 58
after dask 14:54 58
end profile deptht k= 14   14:55 00
begin profile deptht k= 15   14:55 00
after box 14:55 00
after array 14:55 00
after xarray 14:55 00
after stack 14:55 

In [16]:
def check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within NATL60 boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)
        tmin=np.min(timN.values)
        tmax=np.max(timN.values)

        date_proft=pd.to_datetime(date_prof)
        tprof=datetime.datetime(int(date_proft.strftime('%Y')),int(date_proft.strftime('%m')),int(date_proft.strftime('%d')),int(date_proft.strftime('%H')),int(date_proft.strftime('%M')))
        tsec_prof=(tprof-datetime.datetime(1958,1,1,0,0)).total_seconds()


        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) & (tmin < tsec_prof < tmax):
                check=0.
#                print "selected profile falls within NATL60-CJM165 boundaries, the program is proceeding"
        else:
                check=1.
#                print "selected profile does not fall within NATL60-CJM165 boundaries, the program is stopping"
        return check
prof=51
list_profiles = infos.keys()
reference =  str(list_profiles[prof])
lat_prof = int(infos[list_profiles[prof]]['latitude'])
lon_prof = int(infos[list_profiles[prof]]['longitude'])
date_prof = str(infos[list_profiles[prof]]['date'])
file_prof = str(infos[list_profiles[prof]]['file'])
prof_prof = int(infos[list_profiles[prof]]['profile no'])
check=check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
fileEN4 = file_prof
ref_prof=prof_prof

if check == 0.:

    diren4="/media/extra/DATA/EN4/"
    tfileEN4=diren4+fileEN4
    dsen4=xr.open_dataset(tfileEN4)
    laten4=dsen4['LATITUDE'][ref_prof]
    lonen4=dsen4['LONGITUDE'][ref_prof]
    dayen4=dsen4['JULD'][ref_prof]

    dateen4= pd.to_datetime(str(dayen4.values))
    ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
    tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()

    observation_lon=lonen4
    observation_lat=laten4
    observation_time=tsecen4

    tempen4=dsen4['POTM_CORRECTED'][ref_prof]
    salten4=dsen4['PSAL_CORRECTED'][ref_prof]
    depen4=dsen4['DEPH_CORRECTED'][ref_prof]

    observation_dep=depen4
    observation_temp=tempen4
    observation_salt=salten4
    depN = dsN.deptht
    tempN=dsN.votemper
    saltN=dsN.vosaline
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    print dep_level[0]
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    #get the corresponding model level
    model_level=np.zeros(dep_level[0])
    model_dep=np.zeros(dep_level[0])
    obsred_dep=np.zeros(dep_level[0])
    obsred_temp=np.zeros(dep_level[0])
    obsred_salt=np.zeros(dep_level[0])
    for z in np.arange(dep_level[0]):
        obsred_dep[int(z)]=observation_dep[int(z)]
        obsred_temp[int(z)]=observation_temp[int(z)]
        obsred_salt[int(z)]=observation_salt[int(z)]
        diff_dep=0*depN.values
        for k in np.arange(len(depN.values)):
            diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
        lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
        model_level[z]=lev[0]
        model_dep[z]=depN.values[lev[0]]
    #Coarse box in which EN4 profile is contained
    
    step=1
    print "step = 1"
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])
    
    k=0
    model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    %timeit model_tempBOX_array=model_tempBOX.values
    %timeit model_saltBOX_array=model_saltBOX.values
    
    step=2
    print "step = 2"
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])
    
    k=0
    model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    %timeit model_tempBOX_array=model_tempBOX.values
    %timeit model_saltBOX_array=model_saltBOX.values

    step=3
    print "step = 3"
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])
    
    k=0
    model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
    %timeit model_tempBOX_array=model_tempBOX.values
    %timeit model_saltBOX_array=model_saltBOX.values


80.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:79: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:80: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:81: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:82: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-pac

step = 1
The slowest run took 15.38 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 62.7 ms per loop
The slowest run took 21.98 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 51.6 ms per loop
step = 2
1 loop, best of 3: 2.06 s per loop
1 loop, best of 3: 2.27 s per loop
step = 3
1 loop, best of 3: 1.05 s per loop
1 loop, best of 3: 989 ms per loop


In [17]:
def check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within NATL60 boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)
        tmin=np.min(timN.values)
        tmax=np.max(timN.values)

        date_proft=pd.to_datetime(date_prof)
        tprof=datetime.datetime(int(date_proft.strftime('%Y')),int(date_proft.strftime('%m')),int(date_proft.strftime('%d')),int(date_proft.strftime('%H')),int(date_proft.strftime('%M')))
        tsec_prof=(tprof-datetime.datetime(1958,1,1,0,0)).total_seconds()


        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) & (tmin < tsec_prof < tmax):
                check=0.
#                print "selected profile falls within NATL60-CJM165 boundaries, the program is proceeding"
        else:
                check=1.
#                print "selected profile does not fall within NATL60-CJM165 boundaries, the program is stopping"
        return check
prof=51
list_profiles = infos.keys()
reference =  str(list_profiles[prof])
lat_prof = int(infos[list_profiles[prof]]['latitude'])
lon_prof = int(infos[list_profiles[prof]]['longitude'])
date_prof = str(infos[list_profiles[prof]]['date'])
file_prof = str(infos[list_profiles[prof]]['file'])
prof_prof = int(infos[list_profiles[prof]]['profile no'])
check=check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
fileEN4 = file_prof
ref_prof=prof_prof

if check == 0.:

    diren4="/media/extra/DATA/EN4/"
    tfileEN4=diren4+fileEN4
    dsen4=xr.open_dataset(tfileEN4)
    laten4=dsen4['LATITUDE'][ref_prof]
    lonen4=dsen4['LONGITUDE'][ref_prof]
    dayen4=dsen4['JULD'][ref_prof]

    dateen4= pd.to_datetime(str(dayen4.values))
    ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
    tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()

    observation_lon=lonen4
    observation_lat=laten4
    observation_time=tsecen4

    tempen4=dsen4['POTM_CORRECTED'][ref_prof]
    salten4=dsen4['PSAL_CORRECTED'][ref_prof]
    depen4=dsen4['DEPH_CORRECTED'][ref_prof]

    observation_dep=depen4
    observation_temp=tempen4
    observation_salt=salten4
    depN = dsN.deptht
    tempN=dsN.votemper
    saltN=dsN.vosaline
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    print dep_level[0]
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    #get the corresponding model level
    model_level=np.zeros(dep_level[0])
    model_dep=np.zeros(dep_level[0])
    obsred_dep=np.zeros(dep_level[0])
    obsred_temp=np.zeros(dep_level[0])
    obsred_salt=np.zeros(dep_level[0])
    for z in np.arange(dep_level[0]):
        obsred_dep[int(z)]=observation_dep[int(z)]
        obsred_temp[int(z)]=observation_temp[int(z)]
        obsred_salt[int(z)]=observation_salt[int(z)]
        diff_dep=0*depN.values
        for k in np.arange(len(depN.values)):
            diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
        lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
        model_level[z]=lev[0]
        model_dep[z]=depN.values[lev[0]]
    #Coarse box in which EN4 profile is contained
      
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])
    
    k=0
    model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_tempBOX_array=model_tempBOX.values
    model_saltBOX_array=model_saltBOX.values
    model_tBOX_array=model_tBOX.values

    # construction d'un nouveau xarray
    d = {}
    d['time_counter'] = ('time_counter',t_dim)
    d['y'] = ('y',y_dim)
    d['x'] = ('x',x_dim)
    d['nav_lat'] = (['y','x'],model_latBOX_array)
    d['nav_lon'] = (['y','x'],model_lonBOX_array)

    d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
    d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    d['time_counter'] = (['time_counter'], model_tBOX_array)
    dset = xr.Dataset(d)

    latB = dset.nav_lat
    lonB = dset.nav_lon
    model_temperatureB = dset.votemper
    model_salinityB = dset.vosaline
    model_timeB = dset.time_counter

    # selection plus fine des profils
    print "step=1"
    %timeit lon_stacked = lonB[0::1,0::1].stack(profile=('x', 'y'))
    %timeit lat_stacked = latB[0::1,0::1].stack(profile=('x', 'y'))
    print "step=2"
    %timeit lon_stacked = lonB[0::2,0::2].stack(profile=('x', 'y'))
    %timeit lat_stacked = latB[0::2,0::2].stack(profile=('x', 'y'))
    print "step=3"
    %timeit lon_stacked = lonB[0::3,0::3].stack(profile=('x', 'y'))
    %timeit lat_stacked = latB[0::3,0::3].stack(profile=('x', 'y'))


80.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:79: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:80: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:81: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:82: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-pac

step=1
1000 loops, best of 3: 629 µs per loop
1000 loops, best of 3: 630 µs per loop
step=2
1000 loops, best of 3: 542 µs per loop
1000 loops, best of 3: 541 µs per loop
step=3
1000 loops, best of 3: 522 µs per loop
1000 loops, best of 3: 524 µs per loop


In [20]:
def process(jsonfile,infos,prof,dsN,latN,lonN,timN,step):
        ''' Make all the steps needed to create the final netcdf files for one profile
        '''
        list_profiles = infos.keys()
        reference =  str(list_profiles[prof])
        print 'Processing profile ', reference

        lat_prof = int(infos[list_profiles[prof]]['latitude'])
        lon_prof = int(infos[list_profiles[prof]]['longitude'])
        date_prof = str(infos[list_profiles[prof]]['date'])
        file_prof = str(infos[list_profiles[prof]]['file'])
        prof_prof = int(infos[list_profiles[prof]]['profile no'])

        check=check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
        if check == 0.:
            print time.strftime('%d/%m/%y %H:%M',time.localtime())
            profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time = model_mean_percent_profile(file_prof,prof_prof,dsN,latN,lonN,timN,step)
            print time.strftime('%d/%m/%y %H:%M',time.localtime())

def check_prof_NATL60_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within NATL60 boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)
        tmin=np.min(timN.values)
        tmax=np.max(timN.values)

        date_proft=pd.to_datetime(date_prof)
        tprof=datetime.datetime(int(date_proft.strftime('%Y')),int(date_proft.strftime('%m')),int(date_proft.strftime('%d')),int(date_proft.strftime('%H')),int(date_proft.strftime('%M')))
        tsec_prof=(tprof-datetime.datetime(1958,1,1,0,0)).total_seconds()


        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) & (tmin < tsec_prof < tmax):
                check=0.
#                print "selected profile falls within NATL60-CJM165 boundaries, the program is proceeding"
        else:
                check=1.
#                print "selected profile does not fall within NATL60-CJM165 boundaries, the program is stopping"
        return check


def model_mean_percent_profile(fileEN4,ref_prof,dsN,latN,lonN,timN,step):
    #select NATL60 data at the closest depth and within 0.25 and 15days near the location and date of the ARGO profile
    diren4="/media/extra/DATA/EN4/"
    tfileEN4=diren4+fileEN4

    dsen4=xr.open_dataset(tfileEN4)

    laten4=dsen4['LATITUDE'][ref_prof]
    lonen4=dsen4['LONGITUDE'][ref_prof]
    dayen4=dsen4['JULD'][ref_prof]

    dateen4= pd.to_datetime(str(dayen4.values))
    ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
    tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()

    observation_lon=lonen4
    observation_lat=laten4
    observation_time=tsecen4

    tempen4=dsen4['POTM_CORRECTED'][ref_prof]
    salten4=dsen4['PSAL_CORRECTED'][ref_prof]
    depen4=dsen4['DEPH_CORRECTED'][ref_prof]

    observation_dep=depen4
    observation_temp=tempen4
    observation_salt=salten4

    depN = dsN.deptht
    tempN=dsN.votemper
    saltN=dsN.vosaline

    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    print dep_level[0]
    #get the number of useful levels in EN4 profile
    dep_level=np.zeros(1)

    for k in np.arange(len(observation_dep)):
        if not np.isnan(observation_dep[k]):
            dep_level[0]=k

    #get the corresponding model level
    model_level=np.zeros(dep_level[0])
    model_dep=np.zeros(dep_level[0])
    obsred_dep=np.zeros(dep_level[0])
    obsred_temp=np.zeros(dep_level[0])
    obsred_salt=np.zeros(dep_level[0])
    for z in np.arange(dep_level[0]):
        obsred_dep[int(z)]=observation_dep[int(z)]
        obsred_temp[int(z)]=observation_temp[int(z)]
        obsred_salt[int(z)]=observation_salt[int(z)]
        diff_dep=0*depN.values
        for k in np.arange(len(depN.values)):
            diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
        lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
        model_level[z]=lev[0]
        model_dep[z]=depN.values[lev[0]]

    #Coarse box in which EN4 profile is contained
    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
    model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
    t_dim=np.arange(len(model_tBOX))
    x_dim=np.arange(model_lonBOX_array.shape[1])
    y_dim=np.arange(model_lonBOX_array.shape[0])

    def profile_mean_percent(k):

        #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

        model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
        model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

        model_tempBOX_array=model_tempBOX.values
        model_saltBOX_array=model_saltBOX.values
        model_tBOX_array=model_tBOX.values

        # construction d'un nouveau xarray
        d = {}
        d['time_counter'] = ('time_counter',t_dim)
        d['y'] = ('y',y_dim)
        d['x'] = ('x',x_dim)
        d['nav_lat'] = (['y','x'],model_latBOX_array)
        d['nav_lon'] = (['y','x'],model_lonBOX_array)

        d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
        d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
        d['time_counter'] = (['time_counter'], model_tBOX_array)
        dset = xr.Dataset(d)

        latB = dset.nav_lat
        lonB = dset.nav_lon
        model_temperatureB = dset.votemper
        model_salinityB = dset.vosaline
        model_timeB = dset.time_counter

        # selection plus fine des profils

        lon_stacked = lonB[0::step].stack(profile=('x', 'y'))
        lat_stacked = latB[0::step].stack(profile=('x', 'y'))

        distance_threshold = 0.25
        square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
        is_close_to_observation = square_distance_to_observation < distance_threshold**2

        model_temperature_stacked = model_temperatureB[:,0::step].stack(profile=('x', 'y'))
        model_salinity_stacked = model_salinityB[:,0::step].stack(profile=('x', 'y'))

        model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
        model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
        lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
        lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

        model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
        model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
        model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
        model_salt_dask_concat=dask.array.concatenate(model_salt_dask)

        temp_model_mean = model_temp_dask_concat.mean().compute()
        temp_percentile_10= np.percentile(model_temp_dask_concat,10)
        temp_percentile_90= np.percentile(model_temp_dask_concat,90)
        salt_model_mean = model_salt_dask_concat.mean().compute()
        salt_percentile_10= np.percentile(model_salt_dask_concat,10)
        salt_percentile_90= np.percentile(model_salt_dask_concat,90)

        return lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90

    profil_temp_model_mean=np.zeros(dep_level[0])
    profil_temp_model_percent10=np.zeros(dep_level[0])
    profil_temp_model_percent90=np.zeros(dep_level[0])
    profil_salt_model_mean=np.zeros(dep_level[0])
    profil_salt_model_percent10=np.zeros(dep_level[0])
    profil_salt_model_percent90=np.zeros(dep_level[0])


    for z in np.arange(dep_level[0]):
        lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(model_level[z].astype(int))
        profil_temp_model_mean[z]=temp_model_mean
        profil_temp_model_percent10[z]=temp_percentile_10
        profil_temp_model_percent90[z]=temp_percentile_90
        profil_salt_model_mean[z]=salt_model_mean
        profil_salt_model_percent10[z]=salt_percentile_10
        profil_salt_model_percent90[z]=salt_percentile_90


    return profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time




In [21]:
for prof in np.arange(nb_profilesEN4):
    process(jsonfile,infos,prof,dsN,latN,lonN,timN,1)
    process(jsonfile,infos,prof,dsN,latN,lonN,timN,3)



Processing profile   A20130325-44028
25/10/17 16:01
15.0


/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:92: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:93: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:94: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:96: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/albert/anaconda2/lib/python2.7/site-pac

25/10/17 16:03
Processing profile   A20130325-44028
25/10/17 16:03
15.0
25/10/17 16:04
Processing profile   A20130325-44029
25/10/17 16:04
15.0
25/10/17 16:06
Processing profile   A20130325-44029
25/10/17 16:06
15.0
25/10/17 16:08
Processing profile   A20130330-44595
25/10/17 16:08
15.0
25/10/17 16:10
Processing profile   A20130330-44595
25/10/17 16:10
15.0
25/10/17 16:12
Processing profile   A20130325-44023
25/10/17 16:12
15.0
25/10/17 16:14
Processing profile   A20130325-44023
25/10/17 16:14
15.0
25/10/17 16:16
Processing profile   A20130325-44026
25/10/17 16:16
15.0
25/10/17 16:18
Processing profile   A20130325-44026
25/10/17 16:18
15.0
25/10/17 16:20
Processing profile   A20130325-44027
25/10/17 16:20
15.0
25/10/17 16:22
Processing profile   A20130325-44027
25/10/17 16:22
15.0
25/10/17 16:24
Processing profile   A20130326-06421
25/10/17 16:24
80.0
25/10/17 16:28
Processing profile   A20130326-06421
25/10/17 16:28
80.0
25/10/17 16:31
Processing profile   A20130325-44025
25/10/17 16: